<a href="https://colab.research.google.com/github/ajnovice/SecureAI/blob/master/Federated_Learning_Project_MNSIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install syft


     |████████████████████████████████| 225kB 2.9MB/s 
     |████████████████████████████████| 204kB 19.0MB/s 
     |████████████████████████████████| 1.4MB 43.2MB/s 
     |████████████████████████████████| 256kB 43.7MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 389kB 21.1MB/s 
     |████████████████████████████████| 450kB 14.0MB/s 
     |████████████████████████████████| 276kB 43.1MB/s 
     |████████████████████████████████| 51kB 19.5MB/s 
     |████████████████████████████████| 122kB 45.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      

In [4]:
!pip install numpy

In [0]:
import torch as th
from torch import nn, optim

In [6]:
import syft as sy  
hook = sy.TorchHook(th) 
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")
jon = sy.VirtualWorker(hook,id="jon")

W0716 03:34:15.590824 139628840101760 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0716 03:34:15.612728 139628840101760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [7]:
bob.add_workers([alice,jon])
alice.add_workers([alice,bob])
jon.add_workers([alice,bob])

W0716 03:34:17.390873 139628840101760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0716 03:34:17.393442 139628840101760 base.py:628] Worker jon already exists. Replacing old worker which could cause                     unexpected behavior
W0716 03:34:17.394665 139628840101760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0716 03:34:17.396152 139628840101760 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0716 03:34:17.397110 139628840101760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0716 03:34:17.398350 139628840101760 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:jon #objects:0>

In [8]:
model = nn.Linear(2,1)
th.__version__
model.weight

Parameter containing:
tensor([[0.5854, 0.2821]], requires_grad=True)

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [12]:
bob._objects

{4256434034: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 37846550091: tensor([[1.],
         [1.]], requires_grad=True)}

In [13]:
alice._objects

{3193093995: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 49942595726: tensor([[0.],
         [0.]], requires_grad=True)}

In [14]:
jon._objects

{}

In [0]:
def train():
  for i in range(10):
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)
    bob_optim=optim.SGD(params=bob_model.parameters(), lr=0.1)
    alice_optim=optim.SGD(params=alice_model.parameters(), lr=0.1)
    for k in range(10):
      bob_optim.zero_grad()
      boutput = bob_model(data_bob)
      bloss =((target_bob-boutput)**2).sum()
      bloss.backward()
      bob_optim.step()
      
      alice_optim.zero_grad()
      aoutput = alice_model(data_alice)
      aloss = ((target_alice-aoutput)**2).sum()
      alice_optim.step()
    bob_model.move(jon)
    alice_model.move(jon)
    with th.no_grad():
      model.weight.set_(((alice_model.weight.data + bob_model.weight.data) / 2).get())
      model.bias.set_(((alice_model.bias.data + bob_model.bias.data)/2).get())
#     model.weight.set_(((alice_model.weight.data + bob_model.weight.data)/2).get())
#     model.bias.set_(((alice_model.bias.data + bob_model.bias.data)/2).get())
    print("A loss : ",aloss.get().data)
    print("B loss : ",bloss.get().data)
      

In [31]:
train()

A loss :  tensor(0.7981)
B loss :  tensor(5.6212e-06)
A loss :  tensor(0.7968)
B loss :  tensor(2.7521e-06)
A loss :  tensor(0.7959)
B loss :  tensor(1.3475e-06)
A loss :  tensor(0.7953)
B loss :  tensor(6.5960e-07)
A loss :  tensor(0.7948)
B loss :  tensor(3.2297e-07)
A loss :  tensor(0.7945)
B loss :  tensor(1.5812e-07)
A loss :  tensor(0.7943)
B loss :  tensor(7.7390e-08)
A loss :  tensor(0.7942)
B loss :  tensor(3.7892e-08)
A loss :  tensor(0.7941)
B loss :  tensor(1.8560e-08)
A loss :  tensor(0.7940)
B loss :  tensor(9.0802e-09)
